In [1]:
import napari
from skimage import data
import numpy as np
from skimage import measure, morphology, filters
import pandas as pd

In [2]:
image = data.human_mitosis()
nrows, ncols = image.shape

In [3]:
image1 = image[:nrows//2, :ncols//2]
image2 = image[:nrows//2, ncols//2:]
image3 = image[nrows//2:, :ncols//2]
image4 = image[nrows//2:, ncols//2:]
image_list = [image1, image2, image3, image4]

Create Label Images

In [4]:
labels_list = []
for image in image_list:
    filtered = filters.gaussian(image, 1)
    th = filters.threshold_otsu(filtered)
    binary = filtered > th
    labels = measure.label(binary)
    labels_list.append(labels)

Feature Extraction

In [5]:
tables_list = []
for labels, image, i in zip(labels_list, image_list, range(len(image_list))):
    table = pd.DataFrame(measure.regionprops_table(labels, image, properties=['label', 
                                                                                'area', 
                                                                                'axis_major_length', 
                                                                                'axis_major_length', 
                                                                                'feret_diameter_max', 
                                                                                'orientation', 
                                                                                'solidity', 
                                                                                'eccentricity', 
                                                                                'intensity_mean',
    ]))
    table['frame'] = i+1
    tables_list.append(table)

concat tables

In [6]:
master_table = pd.concat(tables_list)
master_table

Stack labels

In [7]:
labels_stack = np.stack(labels_list)
labels_stack.shape

Open in napari

In [9]:
viewer = napari.Viewer()
viewer.add_labels(labels_stack, features=master_table)

<Labels layer 'labels_stack' at 0x254c1751e20>